In [1]:
from collections import Counter
import numpy as np
from nltk import ngrams

ModuleNotFoundError: No module named 'nltk'

In [2]:
pip install nltk

  Created wheel for nltk: filename=nltk-3.5-py3-none-any.whl size=1434678 sha256=daf1daad3f8df3df80d98166116d653d263616365b9924ec70c01f06f39b3d01
  Stored in directory: c:\users\minhyuklee\appdata\local\pip\cache\wheels\45\6c\46\a1865e7ba706b3817f5d1b2ff7ce8996aabdd0d03d47ba0266
Successfully built nltk
Note: you may need to restart the kernel to use updated packages.


In [3]:
from collections import Counter
import numpy as np
from nltk import ngrams

In [4]:
def simple_count(tokens, n): # tokens: 토큰화된 candidate 문장, n: n-gram의 n
    return Counter(ngrams(tokens, n)) # 문장에서 n-gram을 카운트

In [6]:
candidate = "It is a guide to action which ensures that the military always obeys the commands of the party."
tokens = candidate.split() # 단어 토큰화
result = simple_count(tokens, 1) # 토큰화된 문장, 유니그램의 개수를 구하려면 n=1
print(result)

Counter({('the',): 3, ('It',): 1, ('is',): 1, ('a',): 1, ('guide',): 1, ('to',): 1, ('action',): 1, ('which',): 1, ('ensures',): 1, ('that',): 1, ('military',): 1, ('always',): 1, ('obeys',): 1, ('commands',): 1, ('of',): 1, ('party.',): 1})


In [7]:
candidate = "the the the the the the the"
tokens = candidate.split()
result = simple_count(tokens, 1)
print(result)

Counter({('the',): 7})


In [8]:
def count_clip(candidate, reference_list, n):
    cnt_ca = simple_count(candidate, n)
    # Ca 문장에서 n-gram 카운트
    temp = dict() # simple_count의 출력의 데이터 타입이 dict라서
    
    for ref in reference_list: # 다수의 Ref 문장에 대해서 이하 반복
        cnt_ref = simple_count(ref, n)
        # Ref 문장에서 n-gram 카운트
        
        for n_gram in cnt_ref: # 모든 Ref에 대해서 비교하여 특정 n-gram이 하나의 Ref에 가장 많이 등장한 횟수를 저장
            if n_gram in temp:
                temp[n_gram] = max(cnt_ref[n_gram], temp[n_gram]) # max_ref_count
            else:
                temp[n_gram] = cnt_ref[n_gram]
                
    return {
        n_gram: min(cnt_ca.get(n_gram, 0), temp.get(n_gram, 0)) for n_gram in cnt_ca
        # count_clip=min(count, max_ref_count)
        # 위의 get은 찾고자 하는 n-gram이 없으면 0을 반환.
    }

In [9]:
candidate = 'the the the the the the the'
references = [
    'the cat is on the mat',
    'there is a cat on the mat'
]
result = count_clip(candidate.split(), list(map(lambda ref: ref.split(), references)), 1)
print(result)

{('the',): 2}


In [10]:
def modified_precision(candidate, reference_list, n):
    clip = count_clip(candidate, reference_list, n)
    total_clip = sum(clip.values()) # 분자
    
    ct = simple_count(candidate, n)
    total_ct = sum(ct.values()) # 분모
    
    if total_ct == 0: # n-gram의 n이 커졌을 때 분모가 0이 되는 것을 방지
        total_ct = 1
    
    return (total_clip / total_ct) # 보정된 정밀도
    # count_clip의 합을 분자로 하고 단순 count의 합을 분모로 하면 보정된 정밀도

In [12]:
result = modified_precision(candidate.split(), list(map(lambda ref: ref.split(), references)), 1) # 유니그램이므로 n=1
print(result)

0.2857142857142857


In [20]:
def closest_ref_length(candidate, reference_list): # Ca 길이와 가장 근접한 Ref의 길이를 리턴하는 함수
    ca_len = len(candidate) # ca 길이
    ref_lens = (len(ref) for ref in reference_list) # Ref들의 길이
    closest_ref_len = min(ref_lens, key=lambda ref_len: (abs(ref_len - ca_len), ref_len))
    # 길이 차이를 최소화하는 Ref를 찾아서 Ref의 길이를 리턴
    return closest_ref_len

In [21]:
def brevity_penalty(candidate, reference_list):
    ca_len = len(candidate)
    ref_len = closest_ref_length(candidate, reference_list)
    
    if ca_len > ref_len:
        return 1
    elif ca_len == 0:
    # candidate가 비어있다면 BP=0 --> BLEU=0
        return 0
    else:
        return np.exp(1 - ref_len/ca_len)

In [24]:
def bleu_score(candidate, reference_list, weights = [0.25, 0.25, 0.25, 0.25]):
    bp = brevity_penalty(candidate, reference_list) # 브레버티 패널티, BP
    
    p_n = [modified_precision(candidate, reference_list, n=n) for n, _ in enumerate(weights, start=1)]
    # p1, p2, p3, ... pn
    score = np.sum([w_i * np.log(p_i) if p_i != 0 else 0 for w_i, p_i in zip(weights, p_n)])
    return bp * np.exp(score)

In [25]:
import nltk.translate.bleu_score as bleu


candidate = 'It is a guide to action which ensures that the military always obeys the commands of the party'
references = [
    'It is a guide to action that ensures that the military will forever heed Party commands',
    'It is the guiding principle which guarantees the military forces always being under the command of the Party',
    'It is the practical guide for the army always to heed the directions of the party'    
]

print(bleu_score(candidate.split(), list(map(lambda ref: ref.split(), references))))
print(bleu.sentence_bleu(list(map(lambda ref: ref.split(), references)),candidate.split()))

0.5045666840058485
0.5045666840058485
